In [1]:
import numpy as np
import pandas as pd
import zipfile

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [2]:
ziploc ="E:/School/UU/PATREC/CXR_project/"
imgloc = "E:/School/UU\PATREC/.darwin/datasets/v7-labs/covid-19-chest-x-ray-dataset/images/"
store_loc = "E:/School/UU/PATREC/.darwin/datasets/v7-labs/augmented_loc/"

# zip file locs
train_zf = zipfile.ZipFile(ziploc+'final_train_data.zip') 
val_zf = zipfile.ZipFile(ziploc+'final_val_data.zip') 
test_zf = zipfile.ZipFile(ziploc+'test_data.zip') 
train_df = pd.read_csv(train_zf.open('final_train_data.csv'))
val_df = pd.read_csv(val_zf.open('final_val_data.csv'))
test_df = pd.read_csv(test_zf.open('val_data.csv'))

#augmented image location
train_dir = "E:/School/UU/PATREC/.darwin/datasets/v7-labs/augmented_loc/train"
val_dir = "E:/School/UU/PATREC/.darwin/datasets/v7-labs/augmented_loc/val"

In [3]:

def label(df):
    df["label"] = np.nan
    df.loc[df["type"] == "No Pneumonia", 'label'] = "A"
    df.loc[df["type"] == "Bacterial Pneumonia", 'label'] = "B"
#     df.loc[df["type"] == "Fungal Pneumonia", 'label'] = "C"
    df.loc[df["type"] == "Viral Pneumonia", 'label'] = "D"
    df.loc[df["Covid"] == True, 'label'] = "C"
#     df.loc[df["type"] == "Undefined Pneumonia", 'label'] = "F"
    return df

In [4]:
test = label(test_df)
test_df = test[test['label'].notna()]

## dataloader

In [5]:
# rescale and reproduceabillity 
seed = 4
tf.random.set_seed(seed)
datagenerator =  ImageDataGenerator(rescale= 1/255)

# Make train rescled
train_data = datagenerator.flow_from_dataframe(
        dataframe = train_df,
        directory = store_loc +"train",
        x_col = "ogfilename",
        y_col = "label",
        class_mode = "categorical",
        batch_size = 50,
        seed = seed,
        shuffle = True,
        target_size = (224,224), # changed values for vgg16
        keep_aspect_ratio = True,
        validate_filenames= True)

# Make val data
val_data = datagenerator.flow_from_dataframe(
        dataframe = val_df,
        directory = store_loc +"val",
        x_col = "ogfilename",
        y_col = "label",
        class_mode = "categorical",
        batch_size = 50,
        seed = seed,
        shuffle = True,
        target_size = (224,224), # changed values for vgg16
        keep_aspect_ratio = True,
        validate_filenames= True)

# Make test data
test_data = datagenerator.flow_from_dataframe(
        dataframe = test_df,
        directory = imgloc,
        x_col = "ogfilename",
        y_col = None,
        class_mode = None,
        batch_size = 1,
        seed = seed,
        shuffle = False,
        target_size = (224,224), # changed values for vgg16
        keep_aspect_ratio = True,
        validate_filenames= True)

Found 9984 validated image filenames belonging to 4 classes.
Found 1248 validated image filenames belonging to 4 classes.
Found 641 validated image filenames.


In [17]:
train_data


In [6]:
def get_model(weight="imagenet", include_top_layer= False, input_size =(224,224,3), trainable =False, classes =4, optimizer='sgd'):
    vgg =  VGG16(weights=weight, include_top=include_top_layer, input_shape = input_size)
    model = Sequential()
    model.add(vgg)
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
    model.layers[0].trainable = trainable
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=["accuracy"])
    return model

In [7]:
kears_estimator = KerasClassifier(build_fn=get_model, verbose=0)

In [9]:
optimizers = ['rmsprop', 'adam']

param_grid = dict(optimizer=optimizers)

In [12]:
grid = GridSearchCV(estimator=kears_estimator, param_grid=param_grid, cv =3, return_train_score= True, n_jobs=1)

In [19]:
grid_result = grid.fit(train_data, train_df.label[:200])

ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jensr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jensr\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\jensr\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py", line 762, in fit
    self._fit(
  File "C:\Users\jensr\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py", line 918, in _fit
    